In [ ]:
!pip install tensorflow faiss-cpu pillow pandas matplotlib

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
import faiss
import sys


In [ ]:
!git clone https://github.com/Mrunal27/ml_layout_recommender_tf.git
%cd ml_layout_recommender_tf

In [ ]:
sys.path.append('./src')
sys.path.append('./fastapi_app')

In [ ]:
from models import build_embedding_model
from model_utils import LayoutRecommender

In [ ]:
os.makedirs('models', exist_ok=True)
os.makedirs('uploads', exist_ok=True)

In [ ]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
path_to_zip = tf.keras.utils.get_file('flower_photos.tgz', origin=dataset_url, extract=True)
extracted_dir = os.path.join(os.path.dirname(path_to_zip), 'flower_photos')

image_files = glob.glob(os.path.join(extracted_dir, "*/*.jpg"))[:20]
print(f"Number of images for demo: {len(image_files)}")

In [ ]:
df = pd.DataFrame({'image_path': image_files})
df.to_csv('sample_data.csv', index=False)

In [ ]:
embedding_model = build_embedding_model(embedding_dim=128)
embedding_model.save('models/embedding_extractor.keras')  # Safe save
embedding_model = tf.keras.models.load_model('models/embedding_extractor.keras')  # Load back


In [ ]:
embeddings = []
for img_path in df['image_path']:
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (224,224))
    img = tf.keras.applications.efficientnet.preprocess_input(img)
    emb = embedding_model(np.expand_dims(img.numpy(), axis=0))
    embeddings.append(emb.numpy()[0])
embeddings = np.vstack(embeddings)

In [ ]:
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings.astype('float32'))
faiss.write_index(index, 'models/gallery.index')
np.save('models/gallery_paths.npy', np.array(df['image_path']))

In [ ]:
recommender = LayoutRecommender(
    embedding_model_dir='models/embedding_extractor.keras',
    index_path='models/gallery.index',
    gallery_paths_path='models/gallery_paths.npy'
)

In [ ]:
query_img = image_files[0]  # pick first image as query
top_k = 3
recommended_paths = recommender.recommend(query_img, top_k=top_k)

In [ ]:
def show_images(query, recs):
    fig, axes = plt.subplots(1, len(recs)+1, figsize=(12,4))
    axes[0].imshow(Image.open(query))
    axes[0].set_title('Query')
    axes[0].axis('off')
    
    for i, path in enumerate(recs):
        axes[i+1].imshow(Image.open(path))
        axes[i+1].set_title(f'Rec {i+1}')
        axes[i+1].axis('off')
    plt.show()

show_images(query_img, recommended_paths)